In [3]:
with open('secret.txt', 'r') as file:
    content = file.read()
    file.close()
    
client_id, client_secret = content.split()

In [4]:
import requests

token_endpoint = 'https://icdaccessmanagement.who.int/connect/token'
client_id = client_id
client_secret = client_secret
scope = 'icdapi_access'
grant_type = 'client_credentials'


# get the OAUTH2 token

# set data to post
payload = {'client_id': client_id, 
	   	   'client_secret': client_secret, 
           'scope': scope, 
           'grant_type': grant_type}
           
# make request
r = requests.post(token_endpoint, data=payload, verify=False).json()
token = r['access_token']


# access ICD API

url = 'https://id.who.int/icd/entity'

# HTTP header fields to set
headers = {'Authorization':  'Bearer '+token, 
           'Accept': 'application/json', 
           'Accept-Language': 'en',
	   'API-Version': 'v2'}
           
# make request           
r = requests.get(url, headers=headers, verify=False)

# print the result
print (r.text)			

/Users/marco/Documents/python_projects/vector-database-ICD/venv_icd/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'icdaccessmanagement.who.int'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/marco/Documents/python_projects/vector-database-ICD/venv_icd/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'id.who.int'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{"@context":"http://id.who.int/icd/contexts/contextForTopLevel.json","@id":"http://id.who.int/icd/entity","title":{"@language":"en","@value":"WHO Family of International Classifications Foundation"},"releaseId":"2025-01","availableLanguages":["ar","cs","en","es","fr","kk","la","pt","ru","sk","sv","tr","uz","zh"],"releaseDate":"2025-01-24","child":["http://id.who.int/icd/entity/448895267","http://id.who.int/icd/entity/1405434703"],"browserUrl":"https://icd.who.int/browse/2025-01/foundation/en"}


In [15]:
import requests
import json
import os
from typing import Dict, List, Any
import time

class ICD11ApiClient:
    """
    A client for interacting with the ICD-11 API
    """
    def __init__(self, client_id: str, client_secret: str):
        self.client_id = client_id
        self.client_secret = client_secret
        self.base_url = "https://id.who.int/icd/entity"
        self.token = None
        self.token_expiry = 0
        
    def _get_token(self) -> str:
        """
        Get an OAuth token for the ICD API
        """
        if self.token and time.time() < self.token_expiry:
            return self.token
            
        url = "https://icdaccessmanagement.who.int/connect/token"
        payload = {
            "client_id": self.client_id,
            "client_secret": self.client_secret,
            "scope": "icdapi_access",
            "grant_type": "client_credentials"
        }
        headers = {
            "Content-Type": "application/x-www-form-urlencoded"
        }
        
        response = requests.post(url, data=payload, headers=headers)
        response.raise_for_status()
        
        token_data = response.json()
        self.token = token_data["access_token"]
        self.token_expiry = time.time() + token_data["expires_in"] - 60  # Buffer of 60 seconds
        
        return self.token
    
    def get_entity(self, entity_id: str) -> Dict[str, Any]:
        """
        Get details for a specific ICD entity
        """
        token = self._get_token()
        url = f"{self.base_url}/{entity_id}"
        
        headers = {
            "Authorization": f"Bearer {token}",
            "Accept": "application/json",
            "API-Version": "v2",
            "Accept-Language": "en"
        }
        
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        
        return response.json()
    
    def search_entities(self, query: str, limit: int = 10) -> List[Dict[str, Any]]:
        """
        Search for ICD entities based on a query string
        """
        token = self._get_token()
        url = "https://id.who.int/icd/release/11/2023-01/mms/search"
        
        params = {
            "q": query,
            "limit": limit
        }
        
        headers = {
            "Authorization": f"Bearer {token}",
            "Accept": "application/json",
            "API-Version": "v2",
            "Accept-Language": "en"
        }
        
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()
        
        return response.json().get("destinationEntities", [])

# Example usage
client_id = client_id
client_secret = client_secret

if not client_id or not client_secret:
    print("Please set ICD_CLIENT_ID and ICD_CLIENT_SECRET environment variables")
    exit(1)

client = ICD11ApiClient(client_id, client_secret)

# Search for diabetes
results = client.search_entities("pain")
print(f"Found {len(results)} results for 'diabetes'")

for result in results[:10]:  # Show first 3 results
    print(f"Title: {result['title']}")
    print(f"ID: {result['id']}")
    print(f"Definition: {result.get('definition', 'No definition available')}")
    print("---")

Found 50 results for 'diabetes'
Title: <em class='found'>Pain</em>, unspecified
ID: http://id.who.int/icd/release/11/2023-01/mms/661232217/unspecified
Definition: No definition available
---
Title: <em class='found'>Pain</em> disorders, unspecified
ID: http://id.who.int/icd/release/11/2023-01/mms/292232825/unspecified
Definition: No definition available
---
Title: Acute <em class='found'>pain</em>, unspecified
ID: http://id.who.int/icd/release/11/2023-01/mms/1404135736/unspecified
Definition: No definition available
---
Title: Chronic <em class='found'>pain</em>, unspecified
ID: http://id.who.int/icd/release/11/2023-01/mms/1581976053/unspecified
Definition: No definition available
---
Title: Myalgia
ID: http://id.who.int/icd/release/11/2023-01/mms/415195404
Definition: No definition available
---
Title: <em class='found'>Pain</em> in joint
ID: http://id.who.int/icd/release/11/2023-01/mms/9272848
Definition: No definition available
---
Title: Dysmenorrhoea
ID: http://id.who.int/icd/rele

In [19]:
import requests
import csv

# Configuration
TOKEN_URL = "https://icdaccessmanagement.who.int/connect/token"
API_BASE_URL = "https://id.who.int/icd/entity"
CLIENT_ID = client_id  # Replace with your client ID
CLIENT_SECRET = client_secret  # Replace with your client secret
OUTPUT_FILE = "output.csv"  # File to save conversation_id and response_id

def get_access_token(client_id, client_secret):
    """
    Obtain an access token using OAuth2 client credentials.
    """
    payload = {
        "client_id": client_id,
        "client_secret": client_secret,
        "grant_type": "client_credentials",
        "scope": "icdapi_access",
    }
    response = requests.post(TOKEN_URL, data=payload)
    if response.status_code == 200:
        return response.json()["access_token"]
    else:
        raise Exception(f"Failed to obtain access token: {response.status_code}, {response.text}")

def fetch_icd_data(access_token, entity_url):
    """
    Fetch ICD data for a specific entity using the API.
    """
    headers = {
        "Authorization": f"Bearer {access_token}",
        "API-Version": "v2",
        "Accept": "application/json",
        "Accept-Language": "en",
    }
    response = requests.get(entity_url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Failed to fetch data: {response.status_code}, {response.text}")

def process_icd_data(data):
    """
    Process ICD data to extract conversation_id and response_id.
    """
    conversation_id = data.get("@id", "").split("/")[-1]  # Extract ID from @id field
    response_ids = [child.split("/")[-1] for child in data.get("child", [])]  # Extract IDs from child field
    return conversation_id, response_ids

def write_to_csv(conversation_id, response_ids, output_file):
    """
    Write conversation_id and response_ids to a CSV file.
    """
    with open(output_file, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["conversation_id", "response_id"])  # Header row
        for response_id in response_ids:
            writer.writerow([conversation_id, response_id])


try:
    # Step 1: Get access token
    print("Obtaining access token...")
    access_token = get_access_token(CLIENT_ID, CLIENT_SECRET)

    # Step 2: Fetch ICD data for a specific entity (example: root entity)
    print("Fetching ICD data...")
    icd_data = fetch_icd_data(access_token, API_BASE_URL)

    # Step 3: Process ICD data
    print("Processing ICD data...")
    conversation_id, response_ids = process_icd_data(icd_data)

    # Step 4: Write results to CSV file
    print(f"Writing results to {OUTPUT_FILE}...")
    write_to_csv(conversation_id, response_ids, OUTPUT_FILE)

    print("Done! Results saved.")

except Exception as e:
    print(f"Error: {e}")


Obtaining access token...
Fetching ICD data...
Processing ICD data...
Writing results to output.csv...
Done! Results saved.


In [21]:
import requests
import os

# Configuration
TOKEN_URL = "https://icdaccessmanagement.who.int/connect/token"
API_BASE_URL = "https://id.who.int/icd/entity"  # Root entity URL

def get_access_token(client_id, client_secret):
    """
    Obtain an access token using OAuth2 client credentials.
    """
    payload = {
        "client_id": client_id,
        "client_secret": client_secret,
        "grant_type": "client_credentials",
        "scope": "icdapi_access",
    }
    response = requests.post(TOKEN_URL, data=payload)
    if response.status_code == 200:
        return response.json()["access_token"]
    else:
        raise Exception(f"Failed to obtain access token: {response.status_code}, {response.text}")

def fetch_icd_data(access_token, entity_url):
    """
    Fetch ICD data for a specific entity using the API.
    """
    headers = {
        "Authorization": f"Bearer {access_token}",
        "API-Version": "v2",
        "Accept": "application/json",
        "Accept-Language": "en",
    }
    response = requests.get(entity_url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Failed to fetch data: {response.status_code}, {response.text}")

def get_root_categories(access_token):
    """
    Fetch and extract root categories from the ICD Foundation Component.
    """
    root_entity_url = API_BASE_URL  # This is the top-level entity URL
    data = fetch_icd_data(access_token, root_entity_url)
    root_categories = []
    
    # Assuming the root categories are listed in the "child" field of the root entity
    for child_url in data.get("child", []):
        child_id = child_url.split("/")[-1]
        child_entity_url = f"{API_BASE_URL}/{child_id}"
        child_data = fetch_icd_data(access_token, child_entity_url)
        root_categories.append({
            "id": child_id,
            "title": child_data.get("title", {}).get("@value", ""),
        })
    
    return root_categories

try:
    # Step 1: Get access token
    print("Obtaining access token...")
    access_token = get_access_token(CLIENT_ID, CLIENT_SECRET)

    # Step 2: Fetch root categories
    print("Fetching root categories...")
    root_categories = get_root_categories(access_token)

    # Step 3: Print or save the root categories
    print("Root Categories:")
    for category in root_categories:
        print(f"ID: {category['id']}, Title: {category['title']}")

except Exception as e:
    print(f"Error: {e}")


Obtaining access token...
Fetching root categories...
Root Categories:
ID: 448895267, Title: ICD Entity
ID: 1405434703, Title: ICF Entity
